## 매 시도마다 돌릴 것들

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets --quiet

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
from tqdm import tqdm
from collections import Counter
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoConfig
from scipy.special import softmax
import shutil
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import seaborn as sns
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoProcessor, BlipForConditionalGeneration

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## 전처리

In [2]:
# 텍스트 데이터 로드
texts_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/texts.csv', encoding='unicode_escape')
label_txt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/label.txt', sep='\t', header=0)
label_txt.columns = ['ID', 'Annotator1', 'Annotator2', 'Annotator3']
label_jpg = label_txt.copy()

def pre_text(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

texts_df['Text'] = texts_df['Text'].apply(pre_text)

In [3]:
dic = {'positive': 1, 'neutral':0, 'negative': -1}
def pre_labeltext(text):
    new_text = text.split(",")[0]
    return dic[new_text]
label_txt['Annotator1'] = label_txt['Annotator1'].apply(pre_labeltext)
label_txt['Annotator2'] = label_txt['Annotator2'].apply(pre_labeltext)
label_txt['Annotator3'] = label_txt['Annotator3'].apply(pre_labeltext)
label_txt['label'] = label_txt['Annotator1'] + label_txt['Annotator2'] + label_txt['Annotator3']
label_txt['label'] = label_txt['label']/3
label_txt = label_txt.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [4]:
def pre_labeltext(text):
    new_text = text.split(",")[1]
    return dic[new_text]
label_jpg['Annotator1'] = label_jpg['Annotator1'].apply(pre_labeltext)
label_jpg['Annotator2'] = label_jpg['Annotator2'].apply(pre_labeltext)
label_jpg['Annotator3'] = label_jpg['Annotator3'].apply(pre_labeltext)
label_jpg['label'] = label_jpg['Annotator1'] + label_jpg['Annotator2'] + label_jpg['Annotator3']
label_jpg['label'] = label_jpg['label']/3
label_jpg = label_jpg.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [5]:
# 라벨 데이터와 텍스트 데이터 병합
merged_text = pd.merge(label_txt, texts_df, on='ID')

## 데이터 라벨링-모델 일치율 확인(만족스럽다)

In [6]:
def checksentiment(listtexts, listlabels):
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    batch_size = 16
    sentiment_scores = []

    for i in tqdm(range(0, len(listtexts), batch_size)):
        batch_texts = listtexts[i:i+batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
        encoded_input.to(device)

        with torch.no_grad():
            outputs = model(**encoded_input)

        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        scores = probabilities.cpu().numpy()

        pos_probs = scores[:, 2]  # 긍정 확률
        neg_probs = scores[:, 0]  # 부정 확률
        sentiments = pos_probs - neg_probs  # 감성 점수
        sentiment_scores.extend(sentiments)

    listlabels = np.array(listlabels)
    sentiment_scores = np.array(sentiment_scores)

    # L2 Loss
    l2_loss = np.mean((sentiment_scores - listlabels) ** 2)
    print(f"L2 Loss: {l2_loss}")

    return sentiment_scores

In [7]:
lab = merged_text['label'].tolist()
sentiment_scores = checksentiment(merged_text['Text'].tolist(), lab)
check = 0
for i in range(len(sentiment_scores)):
    if sentiment_scores[i] * lab[i] < 0:
        check+=1
check

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

100%|██████████| 1225/1225 [00:16<00:00, 73.89it/s]

L2 Loss: 0.2285643614866811


1713

## 이미지 설명-감성 분석 점수 확인(학습 전 저성능 확인)

In [ ]:

#자기 전에 켜 둘 것
source_dir = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data'

target_dir = '/content/data'

# 대상 폴더가 없으면 생성
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

loader = label_txt['ID'].tolist()

for filename in tqdm(loader):
    filename_with_ext = f"{filename}.jpg"
    src_file = os.path.join(source_dir, filename_with_ext)
    dst_file = os.path.join(target_dir, filename_with_ext)

    try:
        shutil.copy2(src_file, dst_file)
    except Exception as e:
        print(f"파일 복사 중 오류 발생: {filename_with_ext}, 오류: {e}")


  0%|          | 0/19598 [00:00<?, ?it/s]

In [9]:
merged = merged_text

In [10]:
merged = pd.merge(merged, label_txt, on='ID')

In [11]:
merged = pd.merge(merged, label_jpg, on='ID')

In [12]:
scores = np.array(sentiment_scores)
sim = merged['label_x'] * merged['label_y'] > 0
merged['pred'] = scores
input_df = merged[sim]

In [13]:
input_df.drop([1336], axis=0, inplace=True)

<ipython-input-13-a5e798a59b6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df.drop([1336], axis=0, inplace=True)


In [14]:
input_df.drop([623], axis=0, inplace=True)

<ipython-input-14-ae4d4e9167ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df.drop([623], axis=0, inplace=True)


In [15]:
input_df.drop([12183], axis=0, inplace=True)

<ipython-input-15-43ba43229927>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df.drop([12183], axis=0, inplace=True)


In [16]:
image_dir = '/content/data'
def jpg(num):
    return f"/content/data/{num}.jpg"
input_df['ID'] = input_df['ID'].apply(jpg)

<ipython-input-16-30bbe245632d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['ID'] = input_df['ID'].apply(jpg)


In [17]:
input_df = input_df.astype({'ID':'str'})

In [18]:
input_df = input_df.drop('label_y', axis=1)
input_df = input_df.drop('label_x', axis=1)
input_df = input_df.drop('Unnamed: 2', axis=1)
input_df = input_df.drop('label', axis=1)

In [19]:
input_df = input_df.sample(frac=1).reset_index(drop=True)

In [20]:
train_dataframe = input_df[0:10000].reset_index(drop=True)
val_dataframe = input_df[10000:13000].reset_index(drop=True)
test_dataframe = input_df[13000:15816].reset_index(drop=True)

## fine tuning

In [21]:
from PIL import Image
train_dataset = []
for _, row in tqdm(train_dataframe.iterrows()):
    image_path = row["ID"]
    text = row["Text"]

    try:
        image = Image.open(image_path).convert("RGB")  # 이미지 로드
        train_dataset.append({"image": image, "text": text})  # 이미지와 텍스트 저장
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

# 데이터 확인
print("Dataset length:", len(train_dataset))
print("Sample data:", train_dataset[0])

0it [00:00, ?it/s]

Error loading image /content/data/20928.jpg: [Errno 2] No such file or directory: '/content/data/20928.jpg'
Error loading image /content/data/17777.jpg: [Errno 2] No such file or directory: '/content/data/17777.jpg'
Error loading image /content/data/17130.jpg: [Errno 2] No such file or directory: '/content/data/17130.jpg'
Error loading image /content/data/8913.jpg: [Errno 2] No such file or directory: '/content/data/8913.jpg'
Error loading image /content/data/16955.jpg: [Errno 2] No such file or directory: '/content/data/16955.jpg'
Error loading image /content/data/5720.jpg: [Errno 2] No such file or directory: '/content/data/5720.jpg'
Error loading image /content/data/16381.jpg: [Errno 2] No such file or directory: '/content/data/16381.jpg'
Error loading image /content/data/22506.jpg: [Errno 2] No such file or directory: '/content/data/22506.jpg'
Error loading image /content/data/2832.jpg: [Errno 2] No such file or directory: '/content/data/2832.jpg'
Error loading image /content/data/

510it [00:00, 2554.62it/s]

Error loading image /content/data/21227.jpg: [Errno 2] No such file or directory: '/content/data/21227.jpg'
Error loading image /content/data/19201.jpg: [Errno 2] No such file or directory: '/content/data/19201.jpg'
Error loading image /content/data/19301.jpg: [Errno 2] No such file or directory: '/content/data/19301.jpg'
Error loading image /content/data/18681.jpg: [Errno 2] No such file or directory: '/content/data/18681.jpg'
Error loading image /content/data/20948.jpg: [Errno 2] No such file or directory: '/content/data/20948.jpg'
Error loading image /content/data/6823.jpg: [Errno 2] No such file or directory: '/content/data/6823.jpg'
Error loading image /content/data/5071.jpg: [Errno 2] No such file or directory: '/content/data/5071.jpg'
Error loading image /content/data/3657.jpg: [Errno 2] No such file or directory: '/content/data/3657.jpg'
Error loading image /content/data/22783.jpg: [Errno 2] No such file or directory: '/content/data/22783.jpg'
Error loading image /content/data/

766it [00:00, 2428.80it/s]

...

9503it [00:03, 2499.39it/s]

Error loading image /content/data/8956.jpg: [Errno 2] No such file or directory: '/content/data/8956.jpg'
Error loading image /content/data/22387.jpg: [Errno 2] No such file or directory: '/content/data/22387.jpg'
Error loading image /content/data/8576.jpg: [Errno 2] No such file or directory: '/content/data/8576.jpg'
Error loading image /content/data/5004.jpg: [Errno 2] No such file or directory: '/content/data/5004.jpg'
Error loading image /content/data/12433.jpg: [Errno 2] No such file or directory: '/content/data/12433.jpg'
Error loading image /content/data/6785.jpg: [Errno 2] No such file or directory: '/content/data/6785.jpg'
Error loading image /content/data/9565.jpg: [Errno 2] No such file or directory: '/content/data/9565.jpg'
Error loading image /content/data/21882.jpg: [Errno 2] No such file or directory: '/content/data/21882.jpg'
Error loading image /content/data/14974.jpg: [Errno 2] No such file or directory: '/content/data/14974.jpg'
Error loading image /content/data/1703

10000it [00:04, 2494.97it/s]

Error loading image /content/data/5921.jpg: [Errno 2] No such file or directory: '/content/data/5921.jpg'
Error loading image /content/data/15339.jpg: [Errno 2] No such file or directory: '/content/data/15339.jpg'
Error loading image /content/data/2865.jpg: [Errno 2] No such file or directory: '/content/data/2865.jpg'
Error loading image /content/data/6361.jpg: [Errno 2] No such file or directory: '/content/data/6361.jpg'
Error loading image /content/data/14330.jpg: [Errno 2] No such file or directory: '/content/data/14330.jpg'
Error loading image /content/data/11176.jpg: [Errno 2] No such file or directory: '/content/data/11176.jpg'
Error loading image /content/data/7474.jpg: [Errno 2] No such file or directory: '/content/data/7474.jpg'
Error loading image /content/data/16720.jpg: [Errno 2] No such file or directory: '/content/data/16720.jpg'
Error loading image /content/data/19565.jpg: [Errno 2] No such file or directory: '/content/data/19565.jpg'
Error loading image /content/data/22

IndexError: list index out of range

In [ ]:
val_dataset = []
for _, row in tqdm(val_dataframe.iterrows()):
    image_path = row["ID"]
    text = row["Text"]

    try:
        image = Image.open(image_path).convert("RGB")  # 이미지 로드
        val_dataset.append({"image": image, "text": text})  # 이미지와 텍스트 저장
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

# 데이터 확인
print("Dataset length:", len(val_dataset))
print("Sample data:", val_dataset[0])

In [ ]:
from torch.utils.data import Dataset

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [ ]:
from transformers import AutoProcessor, BlipForConditionalGeneration
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
train = ImageCaptioningDataset(train_dataset, processor)
train_dataloader = DataLoader(train, shuffle=False, batch_size=8)

In [ ]:
val = ImageCaptioningDataset(val_dataset, processor)
val_dataloader = DataLoader(val, shuffle=False, batch_size=8)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.to(device)

In [ ]:
count = 5
min_loss = 1000000

In [ ]:
for epoch in range(50):
    model.train()
    total_loss = 0
    print("Epoch:", epoch)
    for idx, batch in tqdm(enumerate(train_dataloader)):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)

        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
    print('train loss:', total_loss)


    model.eval()
    total_loss = 0
    print("Epoch:", epoch)
    for idx, batch in enumerate(tqdm(val_dataloader)):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)

        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids)

        loss = outputs.loss
        total_loss += loss.item()

    print('val loss:', total_loss)

    if min_loss > total_loss:
        min_loss = total_loss
        count = 5
    else:
        count -= 1
    if count == 0:
        break

## 테스트(폐기)

In [29]:
# 감성 분석 모델 로드 (고정된 상태로 사용)
MODEL_SA = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer_sa = AutoTokenizer.from_pretrained(MODEL_SA)
config_sa = AutoConfig.from_pretrained(MODEL_SA)
model_sa = AutoModelForSequenceClassification.from_pretrained(MODEL_SA)
model_sa.to(device)
model_sa.eval()  # 평가 모드

# 감성 분석 모델의 파라미터를 고정
for param in model_sa.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
# 손실 함수 정의 (예: MSELoss)
criterion = nn.MSELoss()

# 옵티마이저 정의 (BLIP 모델의 파라미터만 업데이트)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [31]:
num_epochs = 100  # 최대 에포크 수를 크게 설정
patience = 5      # 검증 손실이 개선되지 않는 에포크 수 허용치
best_val_loss = float('inf')
epochs_no_improve = 0
early_stop = False

In [40]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, image_ids, labels, image_dir, processor):
        self.image_ids = image_ids
        self.labels = labels
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_ids[idx])
        image = Image.open(image_path).convert('RGB')
        # 이미지 전처리 수행
        pix = self.processor(image, return_tensors='pt')
        # 배치 차원 제거 (processor는 보통 배치 차원을 포함하여 반환)
        pix = {k: v.squeeze(0) for k, v in pix.items()}
        label = self.labels[idx]
        return pix, label

# 데이터셋과 데이터로더 생성
train_dataset = ImageDataset(train_image_ids, train_labels, image_dir, processor)
val_dataset = ImageDataset(val_image_ids, val_labels, image_dir, processor)

batch_size = 32  # 적절한 배치 사이즈 설정
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [37]:
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model.to(device)

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

In [41]:
for epoch in range(num_epochs):
    blip_model.train()
    total_loss = 0.0
    for batch in tqdm(train_dataloader):
        pix, labels = batch
        pix = {k: v.to(device) for k, v in pix.items()}
        labels = labels.to(device).float()

        # BLIP 모델을 통해 캡션 생성
        outputs = blip_model.generate(**pix)
        # 생성된 토큰 ID를 디코딩하여 텍스트로 변환
        captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]

        # 감성 분석 모델을 통해 감성 점수 계산
        inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs_sa = model_sa(**inputs_sa)
        scores = outputs_sa.logits.cpu().numpy()
        probs = softmax(scores, axis=1)
        pos_probs = probs[:, 2]  # 긍정 확률 (모델에 따라 인덱스 확인 필요)
        neg_probs = probs[:, 0]  # 부정 확률
        sentiment_scores = pos_probs - neg_probs  # [batch_size]

        # 감성 점수를 텐서로 변환
        sentiment_scores = torch.tensor(sentiment_scores, dtype=torch.float).to(device)

        # 손실 계산 (예: MSELoss 사용)
        loss = criterion(sentiment_scores, labels) * 50
        loss.requires_grad_(True)
        # 역전파 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 검증 루프
    blip_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            pix, labels = batch
            pix = {k: v.to(device) for k, v in pix.items()}
            labels = labels.to(device).float()

            outputs = blip_model.generate(**pix)
            captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]

            inputs_sa = tokenizer_sa(captions, return_tensors="pt", truncation=True, padding=True).to(device)
            outputs_sa = model_sa(**inputs_sa)
            scores = outputs_sa.logits.cpu().numpy()
            probs = softmax(scores, axis=1)
            pos_probs = probs[:, 2]
            neg_probs = probs[:, 0]
            sentiment_scores = pos_probs - neg_probs
            sentiment_scores = torch.tensor(sentiment_scores, dtype=torch.float).to(device)

            loss = criterion(sentiment_scores, labels) * 50
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    # 조기 종료 체크 및 모델 저장
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(blip_model.state_dict(), 'best_blip_model.pth')
        print("Validation loss improved. Model saved.")
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break

100%|██████████| 362/362 [26:01<00:00,  4.31s/it]


Epoch [1/100], Loss: 24.9462


100%|██████████| 41/41 [02:52<00:00,  4.21s/it]


Validation Loss: 25.5248
Validation loss improved. Model saved.


100%|██████████| 362/362 [26:02<00:00,  4.32s/it]


Epoch [2/100], Loss: 24.9436


100%|██████████| 41/41 [02:52<00:00,  4.20s/it]


Validation Loss: 25.5248
No improvement in validation loss for 1 epoch(s).


  1%|▏         | 5/362 [00:24<29:29,  4.96s/it]


KeyboardInterrupt: 